<a href="https://colab.research.google.com/github/MaggieAppleton/Colab-Notebooks/blob/main/Language_Model_Colab_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Language Model Colab Exploration

Exerimenting with calling LLM APIs in colab and creating prompt chains.

## Basic Setup

Install openai and anthropic with pip

In [ ]:
%pip install openai anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 11.1 MB/s eta 0:00:00


Get API keys from Colab secrets.  
Declare clients for Claude and OpenAI and pass in API keys.

In [ ]:
import openai
import anthropic
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')
anthropic_api_key = userdata.get('ANTHROPIC_API_KEY')

openai_client = openai.OpenAI(api_key=openai_api_key)
claude_client = anthropic.Anthropic(api_key=anthropic_api_key)

print("✅ API clients initialized!")

✅ API clients initialized!


Setup basic functions to call Openai and Claude.  
Pass in prompts and default models.

In [ ]:
def call_openai(prompt, model="gpt-4.1-mini-2025-04-14"):
  try:
    response = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=200
    )
    return response.choices[0].message.content.strip()
  except Exception as e:
    print(f"Error calling OpenAI: {e}")
    return None

def call_claude(prompt, model="claude-3-5-haiku-20241022"):
    """
    Call Anthropic's Claude API with the given prompt
    """
    try:
        response = claude_client.messages.create(
            model=model,
            max_tokens=200,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response.content[0].text.strip()
    except Exception as e:
        return f"Error calling Claude API: {str(e)}"

print("✅ API functions ready!")

✅ API functions ready!


Test prompt for both Claude and OpenAI

In [ ]:
print(f"Claude: {call_claude('What is the capital of France?')}")
print(f"Openai: {call_openai('What is the capital of France?')}")

Claude: The capital of France is Paris. Paris is located in the north-central part of the country on the Seine River and is one of the world's major cultural, artistic, and historical centers.
Openai: The capital of France is Paris.


Calling Claude and returning the full response object as JSON.

To make JSON work in Python, you have to import it, then use `print(response.model_dump_json(indent=2))`

Remember to return the response at the end of a function.

In [ ]:
import json

def get_claude_response():
  try:
    response = claude_client.messages.create(
      model="claude-3-5-haiku-20241022",
      max_tokens=200,
      messages=[
        {"role": "user", "content": "Who are the major thought leaders on consciousness and exercise?"}
      ]
    )
    # Print the response in nicely formatted JSON
    print(response.model_dump_json(indent=2))
    return response
  except Exception as e:
    print(f"Error calling Claude API: {str(e)}")
    return None

# Call the function and assign the returned response to a variable
response = get_claude_response()

{
  "id": "msg_01BcEoSEztu88KvQJD1SKaz5",
  "content": [
    {
      "citations": null,
      "text": "# Thought Leaders on Consciousness and Exercise\n\nThis is an interesting intersection that combines two distinct fields. Some notable figures include:\n\n**Consciousness and Exercise Integration:**\n- Mihaly Csikszentmihalyi (pioneered \"flow state\" research applicable to exercise)\n- Jon Kabat-Zinn (mindfulness-based stress reduction, mindful movement)\n- Ellen Langer (mindfulness psychology with applications to physical activity)\n\n**Exercise Science with Mindfulness Components:**\n- Kelly McGonigal (psychologist exploring exercise and mental wellbeing)\n- Dan Siegel (interpersonal neurobiology with mindful movement applications)\n\n**Movement Practitioners:**\n- Moshe Feldenkrais (Feldenkrais Method connecting movement with awareness)\n- Thomas Hanna (somatics - consciousness of the body from within)\n\nIf you're interested in a specific aspect of this",
      "type": "text"
   

Display the response text in nice markdown.

In [ ]:
from IPython.display import display, Markdown

display(Markdown(response.content[0].text))

# Thought Leaders on Consciousness and Exercise

This is an interesting intersection that combines two distinct fields. Some notable figures include:

**Consciousness and Exercise Integration:**
- Mihaly Csikszentmihalyi (pioneered "flow state" research applicable to exercise)
- Jon Kabat-Zinn (mindfulness-based stress reduction, mindful movement)
- Ellen Langer (mindfulness psychology with applications to physical activity)

**Exercise Science with Mindfulness Components:**
- Kelly McGonigal (psychologist exploring exercise and mental wellbeing)
- Dan Siegel (interpersonal neurobiology with mindful movement applications)

**Movement Practitioners:**
- Moshe Feldenkrais (Feldenkrais Method connecting movement with awareness)
- Thomas Hanna (somatics - consciousness of the body from within)

If you're interested in a specific aspect of this

# LLM Workflows

## 1. Prompt Chaining
Break a task down into substeps. Each step builds on previous results. Valuable in scenarios requiring multi-step reasoning, iterative refinement, or complex information processing.

In [ ]:
from typing import List

def chain(input: str, prompts: List[str]) -> str:
  """Chain LLM calls. Pass results between steps."""
  result = input
  for i, prompt in enumerate(prompts, 1):
    print(f"\n-----Step {i}-----")
    print(f"\nInput: {result}")
    print(f"\nPrompt: {prompt}")
    result = call_claude(f"\n{prompt}\nInput: {result}")
    print(f"\nResult:\n{result}")
  return result

Useful for:
- Document Q&A. First extract relevant quotes, then use those as context to generate an answer
- Review and refine generated answers. Fact-check to reduce hallucinations.
- Debug generated code.
- Multi-step problem solving: identify the problem, make a plan, solve each step sequentially.
- Data processing: extract information from text, then format the information in a specific style.

In [ ]:
# Example usage of the chain function

input = "Margaret Thatcher slashed government benefits, leaving Britain's most vulnerable citizens impoverished and angry."

prompts = [
    "Extract the key nouns and verbs from the following sentence. Return them as a comma-separated list.",
    "Using the keywords provided, write a short, snippy tweet. Use less than 280 characters.",
    "Write an ungenerous reply to this tweet. Use less than 280 characters."
]

# Call the chain function
final_output = chain(input, prompts)


-----Step 1-----

Input: Margaret Thatcher slashed government benefits, leaving Britain's most vulnerable citizens impoverished and angry.

Prompt: Extract the key nouns and verbs from the following sentence. Return them as a comma-separated list.

Result:
Nouns: Margaret Thatcher, government, benefits, Britain, citizens
Verbs: slashed, leaving, impoverished

-----Step 2-----

Input: Nouns: Margaret Thatcher, government, benefits, Britain, citizens
Verbs: slashed, leaving, impoverished

Prompt: Using the keywords provided, write a short, snippy tweet. Use less than 280 characters.

Result:
Thatcher slashed government benefits, leaving Britain's citizens impoverished. Her legacy? A country where the vulnerable suffer while the wealthy celebrate. #ThatcherismFailed

-----Step 3-----

Input: Thatcher slashed government benefits, leaving Britain's citizens impoverished. Her legacy? A country where the vulnerable suffer while the wealthy celebrate. #ThatcherismFailed

Prompt: Write an unge

In [ ]:
Second prompt chain example

## 2. Parallelisation

Multiple LLMs run calls at the same time